```{contents}
```
## State Deserialization

**State deserialization** in LangGraph is the process of **reconstructing the in-memory execution state of a graph from its serialized (stored) representation** so that execution can **resume exactly where it left off**.

It is the foundation for **persistence, recovery, replay, fault tolerance, long-running agents, and production reliability**.

---

### **1. Why State Deserialization Exists**

LLM workflows are:

* Long-running
* Non-deterministic
* Costly to restart
* Often interrupted (crashes, timeouts, human input, deployments)

Without deserialization, **every failure loses the entire workflow**.

Deserialization enables:

| Capability            | Why it matters                  |
| --------------------- | ------------------------------- |
| Resume after crash    | Reliability                     |
| Human-in-the-loop     | Pause and continue later        |
| Long-running agents   | Hours / days execution          |
| Distributed execution | Move execution between machines |
| Audit & replay        | Compliance and debugging        |

---

### **2. Conceptual Model**

```
In-Memory State  →  Serialize  →  Store  →  Deserialize  →  In-Memory State
      (Python)                    (DB)                       (Python)
```

At resume time, LangGraph reconstructs:

* State values
* Current node position
* Execution history
* Control flow metadata

---

### **3. What Exactly Gets Serialized**

| Component         | Description                 |
| ----------------- | --------------------------- |
| Graph State       | User-defined `State` object |
| Execution Pointer | Current node                |
| Pending Tasks     | Scheduled operations        |
| Message History   | LLM & agent messages        |
| Metadata          | Thread ID, timestamps       |
| Checkpoints       | Intermediate snapshots      |
| Reducers          | Merge behavior              |
| Control Data      | Loop counters, limits       |

---

### **4. How Deserialization Works Internally**

When a run resumes:

1. Load serialized blob from checkpoint store
2. Decode JSON / binary payload
3. Reconstruct typed state object
4. Restore execution pointer
5. Restore pending edges and tasks
6. Resume scheduler

This converts persisted data into **live Python objects**.

---

### **5. Minimal Working Example**

```python
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph
from typing import TypedDict

class State(TypedDict):
    count: int

saver = SqliteSaver("state.db")

builder = StateGraph(State)
# define nodes & edges...
graph = builder.compile(checkpointer=saver)

# First run
graph.invoke({"count": 0}, config={"thread_id": "job1"})

# Crash / stop

# Resume later
graph.invoke(None, config={"thread_id": "job1"})
```

Here:

* `state.db` holds serialized state
* Second `invoke` triggers **deserialization**

---

### **6. Serialization Format**

LangGraph uses:

* **JSON** for state payloads
* **Pickle-safe objects** for metadata
* **Deterministic encoders** for control structures

State must therefore be:

* JSON serializable
* Deterministic
* Version-stable

---

### **7. Versioning & Schema Evolution**

State schema changes are handled by:

| Mechanism           | Purpose                |
| ------------------- | ---------------------- |
| Default fields      | Backward compatibility |
| Migration functions | Schema upgrades        |
| State version tags  | Detect incompatibility |

Example migration:

```python
def migrate(old):
    if "steps" not in old:
        old["steps"] = 0
    return old
```

---

### **8. Failure Recovery Guarantees**

| Failure         | Behavior                    |
| --------------- | --------------------------- |
| Node crash      | Resume from last checkpoint |
| Process restart | Full state restore          |
| Network failure | Replay last stable state    |
| Human pause     | Continue after approval     |

---

### **9. Production Implications**

| Feature                | Enabled by Deserialization |
| ---------------------- | -------------------------- |
| Long-running agents    | Yes                        |
| Fault tolerance        | Yes                        |
| Distributed execution  | Yes                        |
| Human-in-the-loop      | Yes                        |
| Regulatory audit       | Yes                        |
| Exactly-once semantics | Yes                        |

---

### **10. Mental Model**

State deserialization turns LangGraph from a script into a **transactional execution engine**:

> **LLM Workflow + Database + Scheduler = Reliable AI System**

### Demonstration

In [2]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver

class State(TypedDict):
    step: int
    text: str


In [3]:
checkpointer = InMemorySaver()

In [4]:
def writer(state: State):
    return {"text": state["text"] + f" → step {state['step']}", "step": state["step"] + 1}

def stopper(state: State):
    if state["step"] >= 3:
        return {"done": True}
    return {"done": False}


In [5]:
builder = StateGraph(State)

builder.add_node("write", writer)
builder.add_node("check", stopper)

builder.set_entry_point("write")
builder.add_edge("write", "check")

builder.add_conditional_edges(
    "check",
    lambda s: END if s["done"] else "write",
    {"write": "write", END: END}
)

graph = builder.compile(checkpointer=checkpointer)


In [6]:
result = graph.invoke({"step": 0, "text": "Start"}, config={"thread_id": "job42"})
print(result)


{'step': 3, 'text': 'Start → step 0 → step 1 → step 2'}
